<a href="https://colab.research.google.com/github/somilasthana/MachineLearningSkills/blob/master/NLP_Vector_Space_Representation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## Download Git

In [0]:
import codecs
from collections import defaultdict
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import scipy
import scipy.spatial.distance
import sys

In [2]:
!git clone https://github.com/cgpotts/cs224u

Cloning into 'cs224u'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 1157 (delta 2), reused 6 (delta 2), pack-reused 1140
Receiving objects: 100% (1157/1157), 29.20 MiB | 9.40 MiB/s, done.
Resolving deltas: 100% (657/657), done.


In [3]:
% cd cs224u

/content/cs224u


In [4]:
!ls 

contextualreps.ipynb		 requirements.txt
evaluation_methods.ipynb	 retrofitting.py
evaluation_metrics.ipynb	 setup.ipynb
fig				 sst_01_overview.ipynb
hw1_wordsim.ipynb		 sst_02_hand_built_features.ipynb
hw2_sst.ipynb			 sst_03_neural_networks.ipynb
hw3_rel_ext.ipynb		 sst.py
hw4_wordentail.ipynb		 test
jupyter_notebook_tutorial.ipynb  tf_autoencoder.py
LICENSE				 tf_model_base.py
nli_01_task_and_data.ipynb	 tf_rnn_classifier.py
nli_02_models.ipynb		 tf_shallow_neural_classifier.py
nli.py				 torch_autoencoder.py
np_autoencoder.py		 torch_model_base.py
np_model_base.py		 torch_rnn_classifier.py
np_rnn_classifier.py		 torch_shallow_neural_classifier.py
np_sgd_classifier.py		 torch_subtree_nn.py
np_shallow_neural_classifier.py  torch_tree_nn.py
np_tree_nn.py			 tutorial_pytorch.ipynb
numpy_tutorial.ipynb		 utils.py
README.md			 vsm_01_distributional.ipynb
rel_ext_01_task.ipynb		 vsm_02_dimreduce.ipynb
rel_ext_02_experiments.ipynb	 vsm_03_retrofitting.ipynb
rel_ext.py			 vsm.py


In [37]:
#!wget http://web.stanford.edu/class/cs224u/data/data.zip
!wget http://web.stanford.edu/class/cs224u/data/data.tgz

--2019-09-10 10:20:52--  http://web.stanford.edu/class/cs224u/data/data.tgz
Resolving web.stanford.edu (web.stanford.edu)... 171.67.215.200
Connecting to web.stanford.edu (web.stanford.edu)|171.67.215.200|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1531647000 (1.4G) [application/x-gzip]
Saving to: ‘data.tgz’

data.tgz            100%[===================>]   1.43G   468KB/s    in 53m 6s  

2019-09-10 11:13:58 (470 KB/s) - ‘data.tgz’ saved [1531647000/1531647000]



In [0]:
!gunzip /content/data.tgz

In [39]:
!tar xvf /content/data.tar

data/
data/._.DS_Store
data/.DS_Store
data/._trees
data/trees/
data/._negotiate
data/negotiate/
data/nlidata/
data/wordsim/
data/vsmdata/
data/._rel_ext_data
data/rel_ext_data/
data/._README.md
data/README.md
data/glove.6B/
data/glove.6B/._glove.6B.300d.txt
data/glove.6B/glove.6B.300d.txt
data/glove.6B/._glove.6B.100d.txt
data/glove.6B/glove.6B.100d.txt
data/glove.6B/._glove.6B.50d.txt
data/glove.6B/glove.6B.50d.txt
data/glove.6B/._glove.6B.200d.txt
data/glove.6B/glove.6B.200d.txt
data/rel_ext_data/corpus.tsv.gz
data/rel_ext_data/kb.tsv.gz
data/vsmdata/giga-wordcounts.pickle
data/vsmdata/imdb-wordcounts.pickle
data/vsmdata/imdb_window20-flat.csv.gz
data/vsmdata/._gigawordnyt-advmod-matrix.csv.gz
data/vsmdata/gigawordnyt-advmod-matrix.csv.gz
data/vsmdata/imdb_window5-scaled.csv.gz
data/vsmdata/.ipynb_checkpoints/
data/vsmdata/giga_window5-scaled.csv.gz
data/vsmdata/giga_window20-flat.csv.gz
data/vsmdata/.ipynb_checkpoints/Untitled-checkpoint.ipynb
data/wordsim/MTURK-771.csv
data/wordsim

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -a /content/data  "/content/drive/My Drive/Stanford-Data"

In [0]:
#!pip install vsm

In [0]:
%matplotlib inline
import numpy as np
import os
import pandas as pd
import vsm

In [0]:
DATA_HOME = os.path.join("/content/drive/My Drive/Stanford-Data", 'vsmdata')

In [46]:
DATA_HOME

'/content/drive/My Drive/Stanford-Data/vsmdata'

In [0]:
# Code from https://github.com/cgpotts/cs224u/blob/master/vsm.py

In [0]:
def euclidean(u, v):
    return scipy.spatial.distance.euclidean(u, v)

In [0]:
def observed_over_expected(df):
    col_totals = df.sum(axis=0)
    total = col_totals.sum()
    row_totals = df.sum(axis=1)
    expected = np.outer(row_totals, col_totals) / total
    oe = df / expected
    return oe

In [0]:
def neighbor(df, x):
  if x not in df:
    np.array([])
    
  w = df[x]
  
  dist = df.apply(lambda m: euclidean(w, m), axis =1)
  return dist.sort_values()[:5]

In [0]:
def pmi(df, positive=True):
  df = observed_over_expected(df)
  with np.errstate(divide='ignore'):
    df = np.log(df)
  df[np.isinf(df)] = 0.0
  if positive:
    df[df < 0] = 0.0
  return df

In [0]:
def tfdif(df):
  doccount = float(df.shape[1])
  freqs = df.astype(bool).sum(axis = 1)
  idfs= np.log(doccount / freqs)
  idfs[np.ininf(idfs)] = 0.0
  col_totals = df.sum(axis=0)
  tfs  = df / col_totals
  return (tfs * idfs).T

In [0]:
imdb5 = pd.read_csv(
    os.path.join(DATA_HOME, 'imdb_window5-scaled.csv.gz'), index_col=0)

In [76]:
freq = imdb5_pmi.astype(bool).sum(axis = 1)
freq.head()

!       262
);     1145
.        57
..      340
...     263
dtype: int64

In [80]:
doccount = float(imdb5_pmi.shape[1])
idfs= np.log(doccount / freq)
idfs.head()

!      2.948849
);     1.474033
.      4.474142
..     2.688248
...    2.945039
dtype: float64

In [50]:
imdb5.head()

,!,);,.,..,...,:),?,abandoned,abdomen,abduct,ability,able,abortion,abraham,absence,absolute,absolutely,absorb,abstain,abstract,absurd,abundance,abuse,academy,accelerate,accent,accept,access,accident,accommodation,accomplish,according,account,accumulate,accurate,accuse,ache,achieve,acknowledge,acoustic,...,wrist,write,writer,writers,writing,written,wrong,wrote,x,x-men,yacht,yale,yard,yarn,yawn,yeah,year,year-old,yearn,years,yell,yellow,yen,yes,yet,yield,york,young,younger,youth,zebra,zero,zinc,zombie,zombies,zone,zoo,,,´
!,2.042897e+06,484.716667,5.900860e+04,2663.000000,16021.083333,1693.716667,39386.566667,23.883333,0.866667,1.083333,83.583333,193.666667,22.350000,12.133333,24.566667,561.350000,2731.050000,4.516667,0.50,7.950000,140.050000,11.900000,39.266667,214.650000,3.866667,437.316667,83.383333,13.666667,88.783333,0.700000,21.533333,97.000000,36.550000,0.566667,94.350000,8.500000,15.550000,47.983333,8.116667,1.700000,...,11.700000,297.383333,262.750000,284.466667,507.366667,410.266667,3272.683333,208.883333,168.950000,126.483333,10.633333,2.866667,35.700000,16.200000,204.016667,2080.466667,1728.133333,58.533333,2.350000,2770.833333,82.916667,54.166667,12.300000,3803.616667,1784.366667,3.566667,206.450000,775.400000,135.283333,71.766667,7.650000,195.350000,0.166667,361.816667,388.133333,63.733333,21.966667,605.133333,126.500000,334.600000
);,4.847167e+02,18749.933333,2.730167e+02,5.166667,60.033333,2.500000,262.900000,1.166667,0.000000,0.166667,2.966667,2.000000,0.000000,0.700000,1.400000,2.033333,5.750000,0.000000,0.00,0.700000,2.650000,0.166667,0.700000,2.316667,0.000000,11.083333,0.916667,0.166667,2.866667,0.000000,0.200000,2.433333,1.000000,0.000000,1.566667,0.333333,0.000000,0.200000,0.000000,0.000000,...,0.333333,0.833333,6.833333,2.400000,6.800000,6.700000,5.450000,2.266667,3.866667,4.566667,0.750000,0.000000,1.000000,0.000000,1.950000,2.250000,15.100000,1.783333,0.000000,26.250000,0.500000,1.750000,0.833333,10.416667,38.533333,0.000000,5.783333,35.283333,4.483333,3.066667,0.416667,1.166667,0.000000,2.750000,3.866667,1.283333,0.500000,18.100000,1.916667,0.366667
.,5.900860e+04,273.016667,2.062572e+07,3773.166667,34080.283333,2652.766667,48404.800000,1012.583333,32.083333,27.366667,3016.533333,4890.533333,400.250000,477.416667,840.883333,3232.083333,15387.733333,204.066667,12.45,388.900000,2260.033333,370.600000,1282.966667,2959.166667,18.950000,4310.066667,2051.150000,343.016667,3090.766667,19.383333,840.466667,2062.916667,930.550000,18.683333,2785.016667,104.833333,126.683333,1703.750000,307.683333,48.016667,...,112.400000,3558.316667,6043.050000,4241.200000,11168.000000,10252.916667,25164.666667,3189.283333,2420.850000,1856.883333,133.650000,56.000000,557.116667,367.600000,1232.550000,6982.350000,20526.133333,827.150000,67.216667,43932.133333,198.733333,604.716667,163.716667,26877.833333,27798.050000,70.200000,5451.683333,19014.500000,3852.650000,2121.400000,55.750000,1936.566667,1.583333,3994.983333,3456.016667,1220.266667,308.233333,2799.016667,733.933333,1945.400000
..,2.663000e+03,5.166667,3.773167e+03,126915.900000,1854.983333,116.250000,1700.116667,4.066667,0.000000,0.250000,14.783333,27.000000,2.483333,1.433333,1.700000,20.516667,90.400000,1.483333,0.00,1.566667,10.583333,1.000000,6.183333,9.733333,0.000000,35.683333,13.533333,1.733333,10.000000,0.000000,5.983333,9.500000,3.383333,0.166667,10.700000,0.166667,1.000000,4.916667,1.783333,0.000000,...,0.166667,28.166667,23.033333,22.416667,49.366667,42.266667,191.600000,25.100000,17.066667,8.133333,0.000000,0.250000,3.333333,0.200000,12.616667,254.316667,108.683333,0.333333,0.666667,196.216667,1.700000,3.916667,0.950000,344.116667,216.650000,0.566667,24.883333,94.216667,25.150000,11.750000,0.250000,11.800000,0.000000,34.766667,26.516667,3.566667,2.616667,13.166667,1.983333,20.550000
...,1.602108e+04,60.033333,3.408028e+04,1854.983333,833286.266667,424.383333,10824.833333,29.616667,2.000000,0.733333,73.516667,150.983333,14.650000,7.166667,13.800000,133.600000

In [51]:
len(imdb5.columns)

5000

In [0]:
w = imdb5['good']

In [0]:
dist = imdb5.apply(lambda x: euclidean(w, x), axis=1)

In [60]:
dist.sort_values()[:5]

good      0.000000e+00
job       1.191396e+06
pretty    1.192212e+06
e         1.194603e+06
guys      1.194643e+06
dtype: float64

In [0]:
imdb5_oe = observed_over_expected(imdb5)

In [63]:
imdb5_oe.head()

,!,);,.,..,...,:),?,abandoned,abdomen,abduct,ability,able,abortion,abraham,absence,absolute,absolutely,absorb,abstain,abstract,absurd,abundance,abuse,academy,accelerate,accent,accept,access,accident,accommodation,accomplish,according,account,accumulate,accurate,accuse,ache,achieve,acknowledge,acoustic,...,wrist,write,writer,writers,writing,written,wrong,wrote,x,x-men,yacht,yale,yard,yarn,yawn,yeah,year,year-old,yearn,years,yell,yellow,yen,yes,yet,yield,york,young,younger,youth,zebra,zero,zinc,zombie,zombies,zone,zoo,,,´
!,62.696209,1.457733,0.153929,1.104858,1.024627,5.244205,1.925386,0.120918,0.250424,0.158906,0.133330,0.111218,0.417283,0.244427,0.198968,1.283872,1.445950,0.129100,0.280287,0.126355,0.513408,0.267910,0.191724,0.395123,1.067724,0.787650,0.211198,0.161630,0.238889,0.224538,0.216642,0.387277,0.180000,0.132382,0.243687,0.327630,0.938651,0.200406,0.135879,0.172231,...,0.811620,0.414257,0.247695,0.363681,0.343257,0.237393,1.333127,0.364061,0.472868,0.440614,0.534902,0.316074,0.464897,0.326548,2.964545,3.205615,0.618808,0.301415,0.149776,0.467002,1.939796,0.476665,0.513849,1.801703,0.437683,0.271646,0.200719,0.184750,0.203317,0.251057,0.782219,0.825382,0.322092,0.513553,0.843744,0.366083,0.560435,0.346721,0.329888,0.937063
);,1.457733,5525.667369,0.069789,0.210059,0.376237,0.758532,1.259376,0.578814,0.000000,2.395642,0.463736,0.112550,0.000000,1.381852,1.111116,0.455713,0.298323,0.000000,0.000000,1.090235,0.951964,0.367693,0.334923,0.417888,0.000000,1.956152,0.227519,0.193154,0.755854,0.000000,0.197177,0.952020,0.482593,0.000000,0.396518,1.259037,0.000000,0.081855,0.000000,0.000000,...,2.265899,0.113754,0.631251,0.300674,0.450818,0.379902,0.217550,0.387127,1.060508,1.558904,3.697101,0.000000,1.276097,0.000000,2.776656,0.339726,0.529848,0.899889,0.000000,0.433544,1.146250,1.509086,3.411493,0.483515,0.926204,0.000000,0.550993,0.823801,0.660277,1.051263,4.174947,0.483040,0.000000,0.382494,0.823686,0.722349,1.250046,1.016254,0.489799,0.100626
.,0.153929,0.069789,4.573223,0.133061,0.185262,0.698147,0.201126,0.435750,0.787975,0.341200,0.409001,0.238718,0.635176,0.817476,0.578870,0.628318,0.692480,0.495782,0.593214,0.525381,0.704212,0.709180,0.532447,0.462999,0.444775,0.659828,0.441588,0.344814,0.706872,0.528480,0.718723,0.700068,0.389525,0.370991,0.611404,0.343458,0.649984,0.604832,0.437811,0.413489,...,0.662739,0.421315,0.484217,0.460879,0.642217,0.504265,0.871301,0.472468,0.575916,0.549818,0.571456,0.524817,0.616658,0.629820,1.522321,0.914453,0.624734,0.362039,0.364133,0.629361,0.395179,0.452316,0.581343,1.082156,0.579560,0.454452,0.450519,0.385081,0.492152,0.630785,0.484531,0.695478,0.260083,0.481971,0.638579,0.595767,0.668421,0.136315,0.162683,0.463085
..,1.104858,0.210059,0.133061,711.855906,1.603817,4.866004,1.123541,0.278340,0.000000,0.495745,0.318801,0.209616,0.626800,0.390352,0.186134,0.634360,0.647042,0.573178,0.000000,0.336623,0.524497,0.304356,0.408146,0.242216,0.000000,0.868847,0.463400,0.277129,0.363753,0.000000,0.813796,0.512760,0.225253,0.526368,0.373607,0.086847,0.816046,0.277607,0.403596,0.000000,...,0.156299,0.530432,0.293543,0.387437,0.451515,0.330629,1.055124,0.591405,0.645760,0.383031,0.000000,0.372642,0.586824,0.054501,2.478436,5.297441,0.526117,0.023205,0.574412,0.447080,0.537655,0.465949,0.536531,2.203600,0.718413,0.583458,0.327056,0.303477,0.510986,0.555685,0.345579,0.674007,0.000000,0.667114,0.779273,0.276959,0.902506,0.101987,0.069922,0.778028
...,1.024627,0.376237,0.185262,1.603817,111.057437,2.738271,1.102733,0.312473,1.204296,0.224160,0.244384,0.180687,0.569994,0.300860,0.232913,0.636757,0.643201,0.277969,1.109777,0.179959,0.498088,0.172807,0.449223,0.400544,0.546670,0.605850,0.411176,0.254260,0.516702,0.000000,0.388216,0.481041,0.235702,0.405694,0.434085,0.184745,1.387905,0.287074,0.219783,0.313171,...,1.045646,0.605594,0.305546,0.351853,0.556308,0.332363,1.172149,0.470167,0.550791,0.574344,0.391364,0.448048,0.557221,0.445964,6.046632,5.130735,0.493465,0.216051,0.402878,0.512774,0.736157,0

In [65]:
neighbor(imdb5_oe, 'good')

good     2.051999e-13
.        8.308257e+01
br       8.571157e+01
movie    8.625808e+01
film     9.177150e+01
dtype: float64

In [0]:
imdb5_pmi = pmi(imdb5)

In [70]:
imdb5_pmi.head()

,!,);,.,..,...,:),?,abandoned,abdomen,abduct,ability,able,abortion,abraham,absence,absolute,absolutely,absorb,abstain,abstract,absurd,abundance,abuse,academy,accelerate,accent,accept,access,accident,accommodation,accomplish,according,account,accumulate,accurate,accuse,ache,achieve,acknowledge,acoustic,...,wrist,write,writer,writers,writing,written,wrong,wrote,x,x-men,yacht,yale,yard,yarn,yawn,yeah,year,year-old,yearn,years,yell,yellow,yen,yes,yet,yield,york,young,younger,youth,zebra,zero,zinc,zombie,zombies,zone,zoo,,,´
!,4.138301,0.376882,0.000000,0.099717,0.024329,1.657124,0.655127,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.249881,0.368766,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.065529,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.287527,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.086724,1.164904,0.0,0.0,0.0,0.0,0.662583,0.000000,0.00000,0.588732,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
);,0.376882,8.617159,0.000000,0.000000,0.000000,0.000000,0.230616,0.0,0.000000,0.873651,0.0,0.0,0.0,0.323425,0.105365,0.000000,0.000000,0.0,0.000000,0.086393,0.0,0.0,0.0,0.0,0.000000,0.67098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.230347,0.000000,0.0,0.0,0.0,...,0.817972,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.058748,0.443983,1.307549,0.0,0.243806,0.0,1.021247,0.000000,0.0,0.0,0.0,0.0,0.136496,0.411504,1.22715,0.000000,0.0,0.0,0.0,0.0,0.0,0.049993,1.429102,0.0,0.000000,0.0,0.0,0.0,0.223181,0.016123,0.0,0.0
.,0.000000,0.000000,1.520218,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.420236,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.078956,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
..,0.099717,0.000000,0.000000,6.567876,0.472386,1.582273,0.116486,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.053659,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.907628,1.667224,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.790092,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
...,0.024329,0.000000,0.000000,0.472386,4.710048,1.007327,0.097791,0.0,0.185895,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.104159,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.327796,0.0,0.0,0.0,...,0.044634,0.0,0.0,0.0,0.0,0.0,0.158839,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.799501,1.635249,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.697293,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,1.105405,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


In [69]:
neighbor(imdb5_pmi, 'good')

good     6.679414e-14
br       1.074745e+01
.        1.085449e+01
movie    1.161680e+01
great    1.240925e+01
dtype: float64

In [71]:
neighbor(imdb5_pmi, 'movie')

movie      7.174844e-14
br         6.948659e+00
.          7.178614e+00
film       8.916701e+00
however    9.378645e+00
dtype: float64